In [43]:
import pandas as pd
import os
import json

class Probe(object):
    def __init__(self, data_path, knownColumns = False):
        self.data_path = data_path
        ret = self.ensure_file(self.data_path)
        self.data = pd.read_csv(self.data_path, sep='\t', engine='python', header=None, skiprows=1)
        self.label_columns(knownColumns = knownColumns)
        self.cp = self.get_cp()
        
    @staticmethod
    def ensure_file(path):
        if os.path.isfile(path):
            return True
        else:
            print("Error: file {} does not exist".format(path))
            return False
    
    
    
    #################
    ## DATA INGEST ##
    #################
    
    def label_columns(self, knownColumns = False):
        if self.ensure_file('knownHeaders.txt'):
            #print('file exists')
            with open('knownHeaders.txt') as infile:
                allFormats = json.load(infile)
            selected = False
            while selected == False:
                print('Known headers:')
                for key in allFormats.keys():
                    print(key)
                name = input("Choose an existing header or type NO:")
                if name == 'NO':
                    selected = True
                else:
                    try:
                        colDict = allFormats[name]
                        print(colDict)
                        knownColumns=True
                        selected = True
                    except:
                        print('Unknown header.')
        else:
            allFormats = {}
        if knownColumns:
            #print('applying preselected column headers')
            self.known_columns(colDict)
        else:
            colDict = self.unknown_columns()
            #print(colDict)
            name = input("What would you like to save this header as? If you do not want to save, type NO.")
            if name == 'NO':
                pass
            else:
                allFormats[name] = colDict
                with open('knownHeaders.txt', 'w') as outfile:
                    json.dump(allFormats, outfile)
        return None
    
    def unknown_columns(self):
        # query the user to determine which column is which
        colDict = {}
        print(self.data.head())
        cols = {
            'angle': 'angle', 
            'tunnel reference total pressure': 'tunnel_ref_total_pressure', 
            'unit under test total pressure': 'uut_total_pressure',
            'tunnel reference static pressure': 'tunnel_ref_static_pressure',
            'unit under test static pressure 1': 'uut_static_pressure1',
            'unit under test static pressure 2': 'uut_static_pressure2',
            'date': 'date',
            'time': 'time',
            'temperature in F': 'tempF'
        }
        for key in sorted(cols.keys()):
            col = input("Which column number contains the {}?".format(key))
            col = int(col)
            self.data.rename(index=str, columns={col: cols[key]}, inplace=True)
            colDict[col]=cols[key]
            print('')
            print(self.data.head())
        return colDict
    
    def known_columns(self, colDict):
        for key in colDict.keys():
            self.data.rename(index=str, columns={int(key): colDict[key]}, inplace=True)
        return None
    
    #######################
    ## DATA MANIPULATION ##
    #######################
    
    def get_cp(self):
        # group data by angle
        # in each of 5 pressure columns, take average over one angle and 15 sec
        df = self.data[['tunnel_ref_total_pressure', 'uut_static_pressure1', 'uut_static_pressure2', 'uut_total_pressure','tunnel_ref_static_pressure','angle']]
        # normalize angle column
        df['angle'] = df['angle'].apply(self.normalize_angle)

        mean_df = df.groupby('angle').mean()
        #mean_df.head()
        # normalize by drift data
        # take tunnel_ref_total_pressure, uut_total_pressure (unit under test), take average of each,
        # take difference to find offset to apply to all other uut_total_pressure data (for all angles)
        offset_total_pressure = mean_df['tunnel_ref_total_pressure'][mean_df.index=='Drift Data'].values[0] - mean_df['uut_total_pressure'][mean_df.index=='Drift Data'].values[0]
        mean_df['uut_total_pressure'] = mean_df['uut_total_pressure'] + offset_total_pressure
        # compare both static pressures to the tunnel_ref_static_pressure and do the same as above
        offset_static_pressure1 = mean_df['tunnel_ref_static_pressure'][mean_df.index=='Drift Data'].values[0] - mean_df['uut_static_pressure1'][mean_df.index=='Drift Data'].values[0]
        mean_df['uut_static_pressure1'] = mean_df['uut_static_pressure1'] + offset_static_pressure1
        offset_static_pressure2 = mean_df['tunnel_ref_static_pressure'][mean_df.index=='Drift Data'].values[0] - mean_df['uut_static_pressure2'][mean_df.index=='Drift Data'].values[0]
        mean_df['uut_static_pressure2'] = mean_df['uut_static_pressure2'] + offset_static_pressure2
        # find impact pressure = tunnel_total - tunnel_static (qc) (do for each angle, do )
        mean_df['impact_pressure'] = mean_df['tunnel_ref_total_pressure'] - mean_df['tunnel_ref_static_pressure']
        # divide all uut pressure normalized averages (one total, two static) by impact pressure (qc)
        # take difference of ref_total and uut_total (similar for static)      
        mean_df['cp_total_pressure'] = (mean_df['tunnel_ref_total_pressure'] - mean_df['uut_total_pressure'])/mean_df['impact_pressure']
        mean_df['cp_static_pressure1'] = (mean_df['tunnel_ref_static_pressure'] - mean_df['uut_static_pressure1'])/mean_df['impact_pressure']
        mean_df['cp_static_pressure2'] = (mean_df['tunnel_ref_static_pressure'] - mean_df['uut_static_pressure2'])/mean_df['impact_pressure']
        return mean_df
    
    @staticmethod
    def normalize_angle(s):
        #TODO: Make this more extensible to other formats
        if 'Drift Data' in s:
            return 'Drift Data'
        else:
            return int(s.split('ATP  at')[-1].split('deg AOA')[0])
    
    
class AeroRepeatability(object):
    def __init__(self, golden_probe, uut_probe):
        self.golden_probe = golden_probe
        self.uut_probe = uut_probe
    
    def compare_cp(self, origin_bounds = 0.005):
        #TODO
        return
    
        
###
# after have averages compare this way:
#

In [ ]:
# user sequence:
golden_probe_path = input('Path to golden probe data: ')
uut_probe_path = input('Path to uut probe data: ')
AeroRepeatability(golden_probe_path, uut_probe_path)

In [2]:
golden_probe_path = '/home/stoker/aero_analysis/A0251-50kn'
uut_probe_path = '/home/stoker/aero_analysis/SN003-50kn'

In [44]:
golden_probe = Probe(golden_probe_path)


Known headers:
test1
wrong
aerosonic_default
Choose an existing header or type NO:aerosonic_default
{'0': 'date', '10': 'angle', '3': 'tunnel_ref_static_pressure', '6': 'uut_static_pressure1', '7': 'uut_static_pressure2', '1': 'time', '9': 'tempF', '2': 'tunnel_ref_total_pressure', '5': 'uut_total_pressure'}


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [46]:
uut_probe = Probe(uut_probe_path)

Known headers:
test1
wrong
aerosonic_default
Choose an existing header or type NO:aerosonic_default
{'0': 'date', '10': 'angle', '3': 'tunnel_ref_static_pressure', '6': 'uut_static_pressure1', '7': 'uut_static_pressure2', '1': 'time', '9': 'tempF', '2': 'tunnel_ref_total_pressure', '5': 'uut_total_pressure'}


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [47]:
uut_probe.cp

,tunnel_ref_total_pressure,uut_static_pressure1,uut_static_pressure2,uut_total_pressure,tunnel_ref_static_pressure,impact_pressure,cp_total_pressure,cp_static_pressure1,cp_static_pressure2
angle,,,,,,,,,
-20,14.631547,14.578026,14.571663,14.583221,14.570516,0.061031,0.791832,-0.123047,-0.018793
-18,14.631534,14.577771,14.571408,14.583131,14.570452,0.061082,0.792431,-0.119815,-0.015640
-16,14.631534,14.577056,14.571344,14.582835,14.569467,0.062067,0.784613,-0.122277,-0.030235
-14,14.631547,14.577184,14.571395,14.582887,14.569403,0.062144,0.783022,-0.125210,-0.032052
-12,14.630599,14.576099,14.570462,14.582360,14.568302,0.062297,0.774332,-0.125158,-0.034672
-10,14.630535,14.575472,14.570347,14.581950,14.567470,0.063065,0.770393,-0.126878,-0.045613
-8,14.630458,14.575053,14.570257,14.581719,14.567444,0.063014,0.773476,-0.120739,-0.044638
-6,14.630379,14.574936,14.569622,14.581644,14.566848,0.063531,0.767100,-0.127314,-0.043669
-4,14.630130,14.574876,14.569439,14.581499,14.566472,0.063658,0.763937,-0.132021,-0.046606
